Importing Required Libraries

In [1]:
import string
import pandas as pd

import fr_core_news_md
import de_core_news_md

from stop_words import get_stop_words
from nltk.corpus import stopwords

import gensim
from gensim.models import CoherenceModel
from gensim.models import Phrases

import pyLDAvis
import pyLDAvis.gensim_models


/home/joaop/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Preparing files and NLP - Choose appropriate language

In [2]:
# FR
nlp = fr_core_news_md.load(disable=['parser', 'ner'])
stop_words = set(get_stop_words('french')) | set(stopwords.words('french'))
df = pd.read_csv('AdsFullFR3.csv')

In [ ]:
# DE
nlp = de_core_news_md.load(disable=['parser', 'ner'])
stop_words = set(get_stop_words('german')) | set(stopwords.words('german'))
df = pd.read_csv('AdsFullDE3.csv')

Clean Text

In [3]:
def clean_text(text):
  """
  Cleans the text by removing unnecessary punctuation symbols
  """
  delete_dict = {sp_char: '' for sp_char in string.punctuation}
  delete_dict[' '] =' '
  table = str.maketrans(delete_dict)
  text1 = text.translate(table)
  textArr= text1.split()
  text2 = ' '.join([w for w in textArr if ( not w.isdigit() and
                                           ( not w.isdigit() and len(w)>3))])
  return text2.lower()

In [4]:
df['Description'] = df['Description'].apply(clean_text)

Remove Stopwords

In [5]:
def remove_stopwords(text):
  """
  Removes stopwords from a parameter text
  """
  textArr = text.split(' ')
  rem_text = " ".join([i for i in textArr if i not in stop_words])
  return rem_text

In [6]:
df['Description'] = df['Description'].apply(remove_stopwords)

Text to list

In [7]:
text_list = df['Description'].tolist()

In [8]:
print('Test list: ',text_list[2])

Test list:  opérations compagnie assurance décès entière participation fgt0 bénéfices réalises assurance entière têtes assurance temporaire assurance survie assurances mixtes assurances rentes viagères immédiates têtes rentes viagères capitaux différés rentes viagères différées capitaux différés dots enfants plus amples renseignements remise prospectus gratis sadresser lagentgénéral sclirœll stphilippe maison wirtgen luxembourgeois assurés auprès nationale déjà trois réparitions bénfices équivalant versements


Lemmatization

In [9]:
def lemmatization(texts,allowed_postags=['NOUN', 'ADJ', 'VERB']):
	output = []
	for sent in texts:
		doc = nlp(sent)
		output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
	return output

In [11]:
tokenized_ads = lemmatization(text_list)

In [12]:
bigram = Phrases(tokenized_ads, min_count=25)
for idx in range(len(tokenized_ads)):
    for token in bigram[tokenized_ads[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            tokenized_ads[idx].append(token)
print('List[2]: ',tokenized_ads[2])

List[2]:  ['opération', 'compagnie', 'assurance', 'décès', 'entier', 'participation', 'fgt0', 'bénéfice', 'réaliser', 'assurance', 'entier', 'tête', 'assurance', 'temporaire', 'assurance', 'survi', 'assurance', 'mixte', 'assurance', 'rente', 'viager', 'immédiat', 'tête', 'rente', 'viager', 'capital', 'différé', 'rente', 'viager', 'différé', 'capital', 'différé', 'enfant', 'ample', 'renseignement', 'remettre', 'prospectus', 'grati', 'sadresser', 'lagentgénéral', 'maison', 'luxembourgeois', 'assurer', 'national', 'réparitier', 'bénfice', 'équivaloir', 'versement', 'opération_compagnie', 'assurance_décès', 'entier_participation', 'bénéfice_réaliser', 'assurance_entier', 'tête_assurance', 'temporaire_assurance', 'survi_assurance', 'mixte_assurance', 'rente_viager', 'immédiat_tête', 'rente_viager', 'capital_différé', 'rente_viager', 'différé_capital', 'différé_enfant', 'ample_renseignement', 'remettre_prospectus', 'sadresser_lagentgénéral', 'luxembourgeois_assurer', 'équivaloir_versement']


Document Term Frequency conversion

In [13]:
id2word = gensim.corpora.Dictionary(tokenized_ads)
id2word.filter_extremes(no_below=20, no_above=0.5)
corpus = [id2word.doc2bow(rev) for rev in tokenized_ads]

Creating an LDA Object from the Gensim Library

In [14]:
LDA = gensim.models.ldamodel.LdaModel

Initialising LDA model as lda_model (5 topics as default)

In [15]:
lda_model = LDA(corpus=corpus, id2word=id2word,
                num_topics=10, random_state=100,
                chunksize=1000, passes=100,iterations=250)

In [16]:
# Print top topics
lda_model.print_topics()
print('Top topics:',lda_model.top_topics(corpus))

Top topics: [([(0.009035212, 'santé'), (0.008049848, 'nerf'), (0.007857089, 'tasse'), (0.007690646, 'souffrance'), (0.0069398563, 'arabica'), (0.006016322, 'digestion'), (0.0056558293, 'repas'), (0.005421685, 'lestomac'), (0.005110824, 'gastralgie'), (0.005079033, 'remède'), (0.004633199, 'mauvais'), (0.0045205425, 'irritation'), (0.004210777, 'toux'), (0.0042038257, 'guérir'), (0.0040918067, 'tête'), (0.0040906905, 'rein'), (0.004004669, 'revalenta'), (0.003969617, 'constipation'), (0.0039694216, 'névralgie'), (0.003906678, 'opérer')], -0.49738264854062136), ([(0.016621964, 'maison'), (0.01520073, 'vente'), (0.013605085, 'lieu'), (0.012844302, 'notair'), (0.01006228, 'situer'), (0.008962641, 'heure'), (0.008806027, 'jardin'), (0.0082636, 'are'), (0.0076418323, 'terre'), (0.007637101, 'sieur'), (0.0072462545, 'public'), (0.007005537, 'vendre'), (0.006822117, 'relever'), (0.006545693, 'soussigner'), (0.0064850836, 'notaire'), (0.0064360257, 'requête'), (0.006103685, 'sadresser'), (0.005

Perplexity

In [17]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) 


Perplexity:  -8.118164415367433


Coherence

In [18]:
coherence_model_lda = CoherenceModel(model=lda_model,
                                     texts=tokenized_ads, dictionary=id2word ,
                                     coherence='u_mass', processes=0)
coherence_lda = coherence_model_lda.get_coherence()

In [19]:
print('Calculating coherence...')
print('Coherence: ', coherence_lda)

Calculating coherence...
Coherence:  -1.699912817358276


Visualising the Data

In [20]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/home/joaop/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.038603  0.242305       1        1  24.312632
6      0.007283  0.021519       2        1  17.184578
8     -0.017584  0.218510       3        1  10.775920
3     -0.020925 -0.038425       4        1  10.105042
7      0.206106 -0.127305       5        1   9.245559
2      0.091047  0.142625       6        1   8.635130
4     -0.292860 -0.237878       7        1   7.243978
0     -0.088276 -0.087383       8        1   5.600303
1      0.286653 -0.166123       9        1   4.061138
5     -0.132840  0.032155      10        1   2.835720, topic_info=          Term          Freq         Total Category  logprob  loglift
111      heure  19734.000000  19734.000000  Default  30.0000  30.0000
45        prix  21594.000000  21594.000000  Default  29.0000  29.0000
7         bois   9446.000000   9446.000000  Default  28.0000  28.0000
0     adresser  10162.000000  10162.000000  Default  27.0000  27.0000
27       grand  18360.000000  18360.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
1136       eau    353.392723   1699.075663  Topic10  -5.6246   1.9926
474      année    388.163263   4310.028289  Topic10  -5.5308   1.1556
275    premier    367.662196   7761.526815  Topic10  -5.5850   0.5131
4148   famille    315.682933   1240.630564  Topic10  -5.7375   2.1942
128    prendre    343.410310   5629.646039  Topic10  -5.6533   0.7660

[694 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
1639       2  0.819732  abonnement
1639       8  0.082845  abonnement
1639      10  0.096798  abonnement
1211       9  0.996655      acajou
16668      8  0.997666    accident
...      ...       ...         ...
76         7  0.029975           „
76         8  0.059950           „
76        10  0.010104           „
18094      5  0.998328           ♦
19132      5  0.995806         ♦_♦

[1559 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 7, 9, 4, 8, 3, 5, 1, 2, 6])

In [21]:
# Optional feature to save graph as html file
pyLDAvis.save_html(vis, 'lda_fr.html')